# Getting Started

Load papers and do some preprocessing

In [1]:
# ! pip install -r requirements.txt

Processing /home/conda/feedstock_root/build_artifacts/aiobotocore_1661463101257/work
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/home/conda/feedstock_root/build_artifacts/aiobotocore_1661463101257/work'



In [2]:
import asyncio
import pickle
import pandas as pd
import numpy as np

from redis.asyncio import Redis
# from utils.embeddings import Embeddings

In [3]:
# Connect to the redis instance running in your docker stack at redis:6379
# redis_conn = await Redis(host="redis", port="6379")
# redis-16159.c21898.eu-west-1-1.ec2.cloud.rlrcp.com:16159
# redis_conn = await Redis(host="redis", port="6379")
# redis_conn = await Redis(host="redis-16159.c21898.eu-west-1-1.ec2.cloud.rlrcp.com", port="16159")
redis_conn = await Redis(host="redis-16159.c21898.eu-west-1-1.ec2.cloud.rlrcp.com", port="16159", password="BAPmx2ATT6StewWs8YdSJUvcAPKtQ04M")

In [5]:
# dir(redis_conn)

In [7]:
# Load papers dataframe
def read_paper_df() -> pd.DataFrame:
    with open("arxiv_papers_df.pkl", "rb") as f:
        df = pickle.load(f)
    return df

def paper_key(paper_id: str) -> str:
    return f"paper:{paper_id}"

# Function to concurrently load papers into Redis
async def gather_with_concurrency(n, redis_conn, *papers):
    semaphore = asyncio.Semaphore(n)
    async def load_paper(paper):
        async with semaphore:
            paper["cat_embeddings"] = np.array(paper["cat_embeddings"], dtype=np.float32).tobytes()
            await redis_conn.hset(paper_key(paper["id"]), mapping=paper)
    # gather with concurrency
    await asyncio.gather(*[load_paper(p) for p in papers])


In [4]:
# Investigate
# df = read_paper_df()
# df.head()

df = pd.read_csv("df_merged_pca_embeds.csv")
df.head()

,id,title,year,authors,categories,abstract,count_cats,split_categories,categories_combined,cat_embeddings,kmean_clusters
0,0704.0001,Calculation of prompt diphoton production cros...,2007,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",hep-ph,A fully differential calculation in perturba...,1,['High Energy Physics - Phenomenology'],High Energy Physics - Phenomenology,"[0.36575573682785034, -0.06158893555402756, -0...",6
1,0704.0016,Lifetime of doubly charmed baryons,2008,"Chao-Hsi Chang, Tong Li, Xue-Qian Li and Yu-Mi...",hep-ph,"In this work, we evaluate the lifetimes of t...",1,['High Energy Physics - Phenomenology'],High Energy Physics - Phenomenology,"[0.36575573682785034, -0.06158893555402756, -0...",6
2,0704.0029,Understanding the Flavor Symmetry Breaking and...,2007,"Zhan Shu, Xiao-Lin Chen and Wei-Zhen Deng",hep-ph,"In $\XQM$, a quark can emit Goldstone bosons...",1,['High Energy Physics - Phenomenology'],High Energy Physics - Phenomenology,"[0.36575573682785034, -0.06158893555402756, -0...",6
3,0704.0031,Crystal channeling of LHC forward protons with...,2007,"V. M. Biryukov (Serpukhov, IHEP)",hep-ph,"We show that crystal can trap a broad (x, x'...",1,['High Energy Physics - Phenomenology'],High Energy Physics - Phenomenology,"[0.36575573682785034, -0.06158893555402756, -0...",6
4,0704.0032,Probing non-standard neutrino interactions wit...,2007,"A. Esteban-Pretel, R. Tom\`as and J. W. F. Valle",hep-ph,We analyze the possibility of probing non-st...,1,['High Energy Physics - Phenomenology'],High Energy Physics - Phenomenology,"[0.36575573682785034, -0.06158893555402756, -0...",6


In [6]:
df.keys()

Index(['id', 'title', 'year', 'authors', 'categories', 'abstract',
       'count_cats', 'split_categories', 'categories_combined',
       'cat_embeddings', 'kmean_clusters'],
      dtype='object')

In [ ]:
# df["input"] = df.apply(lambda r: r.title + r.abstract, axis=1)
# df.reset_index(drop=True, inplace=True)

In [ ]:
# For this demo we will take a small sample
# df = df.sample(frac=0.1)
# len(df)

## Embedding Creation

To create embeddings/vector representations of the papers, we will use a combination of the paper abstract and title fields and pass through an open source `SentenceTransformer` model (after some light preprocessing).

Everything is wrapped into the `Embeddings` class and `gather_with_concurrency` function below to help make this cleaner.

In [ ]:
# Create Embeddings
embeddings = Embeddings()
vectors = embeddings.make(df.input.to_list(), show_progress=True)

In [20]:
df["cat_embeddings"] = df["cat_embeddings"].map(lambda x: eval(x))

In [19]:
test_df.head()

0    [0.36575573682785034, -0.06158893555402756, -0...
1    [0.36575573682785034, -0.06158893555402756, -0...
2    [0.36575573682785034, -0.06158893555402756, -0...
3    [0.36575573682785034, -0.06158893555402756, -0...
4    [0.36575573682785034, -0.06158893555402756, -0...
Name: cat_embeddings, dtype: object

In [ ]:
df["vector"] = vectors.tolist()

In [15]:
# Convert dataframe to a dict
papers = df.to_dict("records")

In [22]:
# Load papers to Redis
await gather_with_concurrency(50, redis_conn, *papers)

In [16]:
# Check how many items were stored
await redis_conn.dbsize()

717406

In [8]:
# Check a paper
key = paper_key(df.sample(1)["id"].iloc[0])
await redis_conn.hgetall(key)

{b'abstract': b'  The paper is devoted to hyperbolic (generally speaking, non-Lagrangian and\nnonlinear) partial differential systems possessing a full set of differential\noperators that map any function of one independent variable into a symmetry of\nthe corresponding system. We demonstrate that a system has the above property\nif and only if this system admits a full set of formal integrals (i.e.,\ndifferential operators which map symmetries into integrals of the system). As a\nconsequence, such systems possess both direct and inverse Noether operators (in\nthe terminology of a work by B. Fuchssteiner and A.S. Fokas who have used these\nterms for operators that map cosymmetries into symmetries and perform\ntransformations in the opposite direction). Systems admitting Noether operators\nare not exhausted by Euler-Lagrange systems and the systems with formal\nintegrals. In particular, a hyperbolic system admits an inverse Noether\noperator if a differential substitution maps this syst

In [9]:
# print(key)

paper:1511.09418


## RediSearch Index Creation

Now time to create the search index over all of the documents we have now stored in Redis.



### Add search index for categories!! 


In [17]:
import re

# from config import INDEX_NAME
from redis.asyncio import Redis
from redis.commands.search.query import Query
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.field import VectorField
from typing import Optional, Pattern
INDEX_NAME = "papers"


class TokenEscaper:
    """
    Escape punctuation within an input string. Taken from RedisOM Python.
    """
    # Characters that RediSearch requires us to escape during queries.
    # Source: https://redis.io/docs/stack/search/reference/escaping/#the-rules-of-text-field-tokenization
    DEFAULT_ESCAPED_CHARS = r"[,.<>{}\[\]\\\"\':;!@#$%^&*()\-+=~\/ ]"

    def __init__(self, escape_chars_re: Optional[Pattern] = None):
        if escape_chars_re:
            self.escaped_chars_re = escape_chars_re
        else:
            self.escaped_chars_re = re.compile(self.DEFAULT_ESCAPED_CHARS)

    def escape(self, value: str) -> str:
        def escape_symbol(match):
            value = match.group(0)
            return f"\\{value}"

        return self.escaped_chars_re.sub(escape_symbol, value)

class SearchIndexCat:
    """
    SearchIndex is used to wrap and capture all information
    and actions applied to a RediSearch index including creation,
    manegement, and query construction.
    """
    escaper = TokenEscaper()

    async def create_flat(
        self,
        *fields,
        redis_conn: Redis,
        number_of_vectors: int,
        prefix: str,
        distance_metric: str='L2'
    ):
        """
        Create a FLAT aka brute force style index.

        Args:
            redis_conn (Redis): Redis connection object.
            number_of_vectors (int): Count of the number of initial vectors.
            prefix (str): key prefix to use for RediSearch index creation.
            distance_metric (str, optional): Distance metric to use for Vector Search. Defaults to 'L2'.
        """
        vector_field = VectorField(
            "vector",
            "FLAT", {
                "TYPE": "FLOAT32",
                "DIM": 56,
                "DISTANCE_METRIC": distance_metric,
                "INITIAL_CAP": number_of_vectors,
                "BLOCK_SIZE": number_of_vectors
            }
        )
        await self._create(
            *fields,
            vector_field,
            redis_conn=redis_conn,
            prefix=prefix
        )

    async def create_hnsw(
        self,
        *fields,
        redis_conn: Redis,
        number_of_vectors: int,
        prefix: str,
        distance_metric: str='COSINE'
    ):
        """
        Create an approximate NN index via HNSW.

        Args:
            redis_conn (Redis): Redis connection object.
            number_of_vectors (int): Count of the number of initial vectors.
            prefix (str): key prefix to use for RediSearch index creation.
            distance_metric (str, optional): Distance metric to use for Vector Search. Defaults to 'COSINE'.
        """
        vector_field = VectorField(
            "vector",
            "HNSW", {
                "TYPE": "FLOAT32",
                "DIM": 56,
                "DISTANCE_METRIC": distance_metric,
                "INITIAL_CAP": number_of_vectors,
            }
        )
        await self._create(
            *fields,
            vector_field,
            redis_conn=redis_conn,
            prefix=prefix
        )

    async def _create(
        self,
        *fields,
        redis_conn: Redis,
        prefix: str
    ):
        # Create Index
        await redis_conn.ft(INDEX_NAME).create_index(
            fields = fields,
            definition= IndexDefinition(prefix=[prefix], index_type=IndexType.HASH)
        )

    def process_tags(self, categories: list, years: list) -> str:
        """
        Helper function to process tags data. TODO - factor this
        out so it's agnostic to the name of the field.

        Args:
            categories (list): List of categories.
            years (list): List of years.

        Returns:
            str: RediSearch tag query string.
        """
        tag = "("
        if years:
            years = "|".join([self.escaper.escape(year) for year in years])
            tag += f"(@year:{{{years}}})"
        if categories:
            categories = "|".join([self.escaper.escape(cat) for cat in categories])
            if tag:
                tag += f" (@categories:{{{categories}}})"
            else:
                tag += f"(@categories:{{{categories}}})"
        tag += ")"
        # if no tags are selected
        if len(tag) < 3:
            tag = "*"
        return tag

    def vector_query(
        self,
        categories: list,
        years: list,
        search_type: str="KNN",
        number_of_results: int=20
    ) -> Query:
        """
        Create a RediSearch query to perform hybrid vector and tag based searches.


        Args:
            categories (list): List of categories.
            years (list): List of years.
            search_type (str, optional): Style of search. Defaults to "KNN".
            number_of_results (int, optional): How many results to fetch. Defaults to 20.

        Returns:
            Query: RediSearch Query

        """
        # Parse tags to create query
        tag_query = self.process_tags(categories, years)
        base_query = f'{tag_query}=>[{search_type} {number_of_results} @vector $vec_param AS vector_score]'
        return Query(base_query)\
            .sort_by("vector_score")\
            .paging(0, number_of_results)\
            .return_fields("paper_id", "paper_pk", "vector_score")\
            .dialect(2)

    def count_query(
        self,
        years: list,
        categories: list
    ) -> Query:
        """
        Create a RediSearch query to count available documents.

        Args:
            categories (list): List of categories.
            years (list): List of years.

        Returns:
            Query: RediSearch Query
        """
        # Parse tags to create query
        tag_query = self.process_tags(categories, years)
        return Query(f'{tag_query}')\
            .no_content()\
            .dialect(2)



In [20]:
from redis.commands.search.field import (
    TagField,
    VectorField,
    NumericField,
    TextField
)
# from utils.search_index import SearchIndex

# Search index helper class
# search_index = SearchIndexCat("papers", redis_conn)
search_index = SearchIndexCat()

In [26]:
len(df["cat_embeddings"][0])

56

In [23]:
# Create a document schema that includes 3 indexed fields
# --> vector, categories, and year

# vector_field = VectorField(
#     "vector",
#     "HNSW", {
#         "TYPE": "FLOAT32",
#         "DIM": 768,
#         "DISTANCE_METRIC": "COSINE",
#         "INITIAL_CAP": len(papers),
#     }
# )


vector_field = VectorField(
    "cat_embeddings",
    "FLAT", {
        "TYPE": "FLOAT32",
        # "DIM": 768,
        "DIM": 56,
        "DISTANCE_METRIC": "COSINE",
        "INITIAL_CAP": len(papers),
        "BLOCK_SIZE": len(papers)
    }
)
categories_field = TagField("categories")
year_field = TagField("year")

# Create the index with the schema and over documents containing the prefix "paper:"
await search_index.create_flat(
    categories_field,
    year_field,
    vector_field,
    redis_conn=redis_conn,
    number_of_vectors=len(papers),
    prefix="paper:"
)

## Test queries!

Use the [`running_queries.ipynb`](running_queries.ipynb) notebook to test out queries.

In [24]:
async def fetch_one(redis_conn):
    async for key in redis_conn.scan_iter("paper:*"):
        return await redis_conn.hgetall(key)

In [25]:
paper = await fetch_one(redis_conn)

## Insurance Claims Example

The above code works for the demo dataset or arXiv papers! Extending the example to another use case, we can see how this might work for insurance claims and policy data.

In [ ]:
# First delete the other dataset from Redis...
await search_index.delete()

In [ ]:
# Claims Schema
# Here we assume some insurance claims might have a schema like this:

claims_index = SearchIndex("claims", redis_conn)

# Sample dummy data
claims = [{
    "claims_id": "1235",
    "customer_id": "5341345",
    "timestamp_of_incident": 1665765963, # Epoch timestamp
    "timestamp_of_submission": 1666716363,
    "claim_description": "This includes written text that describes the incident from the customer's POV",
    "text_vector": np.random.random(size=786),
    "age_of_customer": 33
}]

In [ ]:
# To work with this data in RediSearch, we need to construct a schema and create an index

# Schema Definitions
customer = TagField("customer_id") # to be able to filter/sort by customer ID
timestamp_of_incident = NumericField("timestamp_of_incident")
timestamp_of_submission = NumericField("timestamp_of_submission")
claim_description = TextField("claim_description")
text_vector = VectorField(
    "text_vector",
    "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": 768,
        "DISTANCE_METRIC": "IP",
        "INITIAL_CAP": len(claims),
        "BLOCK_SIZE": len(claims)
    }
)
age_of_customer = NumericField("age_of_customer")


# Create Index
await claims_index.create(
    customer,
    timestamp_of_incident,
    timestamp_of_submission,
    claim_description,
    age_of_customer,
    text_vector,
    prefix="claim:"
)


In [ ]:
# Similar to above, we also need to load data to Redis

def claim_key(claim_id: str) -> str:
    return f"claim:{claim_id}"

for claim in claims:
    claim["text_vector"] = np.asarray(claim["text_vector"], dtype=np.float32).tobytes()
    await redis_conn.hset(claim_key(claim["claims_id"]), mapping=claim)


In [ ]:
await redis_conn.hgetall(f"claim:{claim['claims_id']}")